# Estimating quartiles

# In brevi

The purpose of this notebook is to explore the ways that Python calculates quartiles. During the development of the anova_one_factor notebook, I discovered that Python, Libre Office, and Excel calculate quartiles in the same way, but that Minitab and an online source calculate them differently. I've discovered that there are at least nine ways to calculate quartiles.

# Data

Download the data file.

[estimating_quartiles](https://drive.google.com/open?id=1Nc_VFXo2SrsSdprfCmQYhLbJawAzKpH6)